In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [2]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'

#start Glue
# app = GlueApplication(dc)

# imageviewer = app.new_data_viewer(ImageViewer)
# imageviewer.add_data(image)

# app.start()



In [3]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [4]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [5]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [6]:
array1 = [1,3,5,6]
array2 = [4,5,6,6]
array3 = [6,7,8,9]

colnames = ['array1', 'array2', 'array3']
rownames = ['col1', 'col2', 'col3', 'col4']

data = np.asarray([array1, array2, array3]).transpose()

pandas_data = pd.DataFrame(data, index=rownames, columns=colnames)

In [43]:
# Full working example! 
# Has a checkbox system for choosing components- does statistics cumulatively
# To do:
# Fix transition from many components to few
# Maintain checked state when user switches data sets
# Make the default "choose a data set" make more sense
# Remove data from the table when user unchecks a data set

import sys
from PyQt5 import QtCore, QtWidgets, QtGui
Qt = QtCore.Qt
from PyQt5.QtCore import QVariant
from qtpy.QtWidgets import (QApplication, QLabel, QWidget, QComboBox, QCheckBox,
                            QVBoxLayout, QPushButton, QGridLayout, QTableView) 

class stat_widget(QWidget):
    # Sets up the stat widget
    def __init__(self, data_collection, parent = None):
        super(stat_widget, self).__init__(parent)
        self.view = QtCore.QAbstractTableModel.__init__(self, parent)
        self.dc = data_collection  
        self.lencomponents = 0
        
        # The array of checkboxes
        self.checks = []        
        
        self.done_pile = []
        
        self.mean_array = []
        self.median_array = []
        self.min_array = []
        self.max_array = []
        self.sum_array = []
        self.name_array = []
        
        # set up vertical layout
        self.layout = QVBoxLayout()
        
        # Dropdown to choose the data
        self.cb_data = QComboBox()
        self.cb_data.addItem('Choose a data set')
        
        for i in range(0, len(self.dc.data)):
            self.cb_data.addItem(self.dc.data[i].label)
        
        self.cb_data.currentIndexChanged.connect(self.dataSelectionChange)
    
        self.layout.addWidget(self.cb_data) 
    
        self.view = QTableView()
        self.layout.addWidget(self.view)

        self.setLayout(self.layout)
        self.setWindowTitle("Statistics")          
     
    def runStats(self, parent=None):
        comp_i = self.index
        
        # Check to see if the stats have already been done
        if comp_i in self.done_pile:
            pass
        else:     
            headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

            name = dc[self.data_index].label + '--' + self.dc[self.data_index].components[comp_i].label
            self.name_array.append(name) # add to the name array to build the table
            self.mean_array.append(self.dc[self.data_index].compute_statistic('mean', self.dc[self.data_index].components[comp_i]))
            self.median_array.append(self.dc[self.data_index].compute_statistic('median', self.dc[self.data_index].components[comp_i]))       
            self.min_array.append(self.dc[self.data_index].compute_statistic('minimum', self.dc[self.data_index].components[comp_i]))      
            self.max_array.append(self.dc[self.data_index].compute_statistic('maximum', self.dc[self.data_index].components[comp_i]))     
            self.sum_array.append(self.dc[self.data_index].compute_statistic('sum', self.dc[self.data_index].components[comp_i]))            
                       
            column_data = np.asarray([self.mean_array, self.median_array, self.min_array, self.max_array, self.sum_array]).transpose()

            self.data_frame = pd.DataFrame(column_data, index=self.name_array, columns=headings)             
        
        # Set up data as pandasModel
            model = pandasModel(self.data_frame)

            self.view.setModel(model)
            self.view.setShowGrid(False)

            self.layout.addWidget(self.view)                                              
        
    def dataSelectionChange(self,i):
        # Linked to the drop down menu
        self.data_index = i-1        
#         self.cb_data.removeItem(0)

        # Clear the widgets from the previous round
        for k in range (0, self.lencomponents):
            self.layout.removeWidget(self.checks[k])
        
        for j in range (0, len(self.dc[self.data_index].components)):
            self.lencomponents = len(self.dc[self.data_index].components)            
            self.checks.append(QCheckBox(self.dc[self.data_index].components[j].label))

            self.checks[j].clicked.connect(self.setIndex)
            self.layout.addWidget(self.checks[j])
    
    def setIndex(self, parent = None):
    # Sets the indices for the newly checked boxes
        for j in range (0, len(self.checks)):
            if self.checks[j].isChecked():
                self.index = j
                self.runStats()
                
                # Add that index to the done pile to make sure it doesn't repeat
                self.done_pile.append(j)
#             else if not self.checks[j].isChecked() and j in self.done_pile:
                    # Ideally remove from the table
    
class pandasModel(QtCore.QAbstractTableModel):
    # Set up the data in a form that allows it to be added to qt widget
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(pandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()                   
            
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
    
ex = stat_widget(dc)
ex.show()
sys.exit(app.exec_())

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x15195ef0d8>


SystemExit: 0

/Users/laurachapman/anaconda/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [42]:
help(QComboBox)

Help on class QComboBox in module PyQt5.QtWidgets:

class QComboBox(QWidget)
 |  QComboBox(parent: QWidget = None)
 |  
 |  Method resolution order:
 |      QComboBox
 |      QWidget
 |      PyQt5.QtCore.QObject
 |      sip.wrapper
 |      PyQt5.QtGui.QPaintDevice
 |      sip.simplewrapper
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __len__(self, /)
 |      Return len(self).
 |  
 |  activated = <unbound PYQT_SIGNAL activated(int)>
 |  addItem(...)
 |      addItem(self, str, userData: Any = None)
 |      addItem(self, QIcon, str, userData: Any = None)
 |  
 |  addItems(...)
 |      addItems(self, Iterable[str])
 |  
 |  changeEvent(...)
 |      changeEvent(self, QEvent)
 |  
 |  clear(...)
 |      clear(self)
 |  
 |  clearEditText(...)
 |      clearEditText(self)
 |  
 |  completer(...)
 |      completer(self) -> QCompleter
 |  
 |  contextMenuEvent(...)
 |      contextMenuEvent(self, QContextMenuEvent)
 |  
 |  count(...)
 |      count(self) -> int
 |  
 |  curren